In [55]:
import websocket
import json
import pandas as pd
from binance import Client
import time
from datetime import datetime, timedelta

In [45]:
api_key = ''
api_secret = ''
client = Client(api_key, api_secret)

In [54]:
csv_file_path = 'orders.csv'

In [46]:
endpoint = 'wss://stream.binance.com:9443/ws'

In [47]:
our_msg = json.dumps({
    "method": "SUBSCRIBE",
    "params": ['btcusdt@kline_1h'],
    "id": 1
})

In [48]:
def gettime():    
    timestart = datetime.now() - timedelta(hours=12)
    timeend = timestart + timedelta(seconds=1)
    return str(timestart), str(timeend)

In [49]:
def get12hprice():
    timestart, timeend = gettime()
    frame = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                                                     '1s',
                                                     timestart,
                                                     timeend))
    value = frame.iloc[:,4:5]
    return float(value.iloc[0,0])

In [50]:
in_position = False

In [66]:
def on_open(ws):
    ws.send(our_msg)

def on_message(ws, message):
    global df, in_position, orderqty, buyprice
    out = json.loads(message)
    thprice = get12hprice()
    df = pd.DataFrame(out['k'],index=[pd.to_datetime(out['E'],
                                            unit='ms')])[['c']]
    df['Price (12h ago)'] = thprice
    df['Diff(%)'] = float(df.c)/float(thprice) -1
    df = df.rename(columns={'c': 'Price(Now)'})
    print(df)
    
    if not in_position and float(df['diff']) >= 0.016:
        buyprice = float(df.c)
        buyorder = buy()
        buyorder.to_csv('allorders.csv',mode='a',header=False)
        in_position = True
        print('bought')
        
    if in_position:        
        if float(df.c) >= buyprice * 1.0084:
            sellorder = sell()
            sellorder.to_csv('allorders.csv',mode='a',header=False)
            in_position = False
            print('profit: '+ str(float(df.c) - buyprice))
        elif float(df.c) <= buyprice * 0.94:
            sellorder = sell()
            sellorder.to_csv('allorders.csv',mode='a',header=False)
            in_position = False
            print('loss: ' + str(float(df.c) - buyprice))

In [65]:
ws = websocket.WebSocketApp(endpoint, on_message=on_message,
                           on_open=on_open)
ws.run_forever()

                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:28:22.629  42164.00000000         44875.0 -0.060412
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:28:24.637  42184.77000000         44875.0 -0.059949
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:28:26.642  42198.00000000        44874.48 -0.059644
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:28:28.663  42212.14000000        44874.02 -0.059319
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:28:30.667  42234.00000000         44872.0 -0.058789
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:28:32.674  42234.01000000        44869.59 -0.058739
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:28:34.723  42220.00000000        44869.59 -0.059051
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03

                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:30:29.936  42274.01000000        44899.97 -0.058485
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:30:31.980  42250.00000000        44899.96 -0.059019
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:30:33.994  42238.00000000        44908.87 -0.059473
                         Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:30:36  42234.00000000        44908.87 -0.059562
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:30:38.067  42228.00000000        44908.87 -0.059696
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:30:40.112  42204.84000000        44911.09 -0.060258
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:30:42.113  42174.00000000        44911.09 -0.060945
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:30:4

                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:32:41.221  42119.83000000        44998.08 -0.063964
                             Price(Now)  Price (12Hago)  Diff(%)
2024-01-03 12:32:43.241  42120.00000000        44998.08 -0.06396
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:32:45.301  42124.45000000        44998.08 -0.063861
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:32:47.307  42118.76000000        44998.08 -0.063988
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:32:51.323  42089.71000000        44998.09 -0.064633
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:32:53.328  42083.81000000        44998.09 -0.064765
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:32:55.342  42094.89000000        44998.08 -0.064518
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 1

                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:34:50.646  42258.00000000         45000.5 -0.060944
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:34:52.710  42259.20000000         45000.5 -0.060917
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:34:54.713  42261.98000000        45000.49 -0.060855
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:34:56.724  42261.99000000         45000.5 -0.060855
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:34:58.730  42259.47000000        45000.49 -0.060911
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:35:00.730  42232.68000000         45000.5 -0.061506
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:35:02.812  42244.64000000         45000.5 -0.061241
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03

                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:36:58.212  42369.52000000         44986.0 -0.058162
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:37:00.282  42374.54000000        44986.01 -0.058051
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:37:02.303  42380.01000000         44986.0 -0.057929
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:37:04.353  42382.10000000        44981.35 -0.057785
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:37:06.360  42407.75000000        44981.35 -0.057215
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:37:08.381  42412.44000000         44981.1 -0.057105
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:37:10.383  42417.51000000         44981.1 -0.056993
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03

                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:39:05.635  42227.61000000        45006.26 -0.061739
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:39:07.644  42225.60000000        45006.25 -0.061784
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:39:09.649  42228.65000000        45006.25 -0.061716
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:39:11.691  42239.09000000        45006.25 -0.061484
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:39:13.697  42249.94000000        45006.26 -0.061243
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:39:15.701  42249.83000000        45006.25 -0.061245
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:39:17.757  42272.99000000        45006.25 -0.060731
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03

                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:41:13.192  42140.87000000        45061.16 -0.064807
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:41:15.216  42131.11000000        45061.16 -0.065024
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:41:17.237  42136.17000000        45058.01 -0.064846
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:41:19.252  42115.20000000        45058.01 -0.065312
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:41:21.268  42116.52000000        45058.01 -0.065282
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:41:23.320  42135.91000000         45058.0 -0.064852
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:41:25.357  42133.08000000        45058.01 -0.064915
                             Price(Now)  Price (12Hago)  Diff(%)
2024-01-03 

                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:43:21.172  42247.98000000        45047.07 -0.062137
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:43:23.231  42230.00000000        45048.33 -0.062562
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:43:25.245  42228.11000000        45048.33 -0.062604
                             Price(Now)  Price (12Hago)   Diff(%)
2024-01-03 12:43:27.354  42228.09000000        45048.34 -0.062605


True

In [42]:
def buy():
    order = client.create_order(symbol='BTC',
                                side='BUY',
                                type='MARKET',
                                qty='quoteOrderQty')
    return order

In [43]:
def sell():
    order = client.create_order(symbol='BTC',
                                side='SELL',
                                type='MARKET',
                                qty='quoteOrderQty')
    return order